# Tavily Search

[Tavily's Search API](https://tavily.com) is a search engine built specifically for AI agents (LLMs), delivering real-time, accurate, and factual results at speed.

## Setup

The integration lives in the `langchain-community` package. We also need to install the `tavily-python` package itself.

In [12]:
pip install --quiet -U langchain-community tavily-python


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Credentials

We also need to set our Tavily API key. You can get an API key by visiting [this site](https://app.tavily.com/sign-in) and creating an account.

In [1]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability

In [2]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Instantiation

Here we show how to instatiate an instance of the Tavily search tools, with 

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(
    max_results=3,
    search_depth="basic",
    include_domains = ["https://langchain.com"],
    exclude_domains = ["https://braintrust.com"],
    include_answer = False,
    include_raw_content = False,
    include_images = False
)

## Invocation

We can invoke the tool in two ways. One is with a normal query, and the other is by providing it with a `ToolCall`. If we invoke it with a normal query, only the content of the tool will be returned. If we invoke it with a `ToolCall`, a `ToolMessage` will be returned.

### Invoke with normal query

In [3]:
ans = tool.invoke({"query": "What happened in the latest burning man floods"})
print(ans)

{'input': 'What happened in the latest burning man floods?', 'output': 'In the latest Burning Man festival, heavy rains caused flooding and resulted in thousands of attendees being stranded. The festival took place in Black Rock Forest, Nevada, and around 70,000 people were gathered for the event., {'input': 'What happened in the latest burning man floods?', 'output': 'The latest Burning Man festival experienced heavy rain, resulting in floods and muddy conditions. Thousands of attendees were stranded at the festival site in Nevada. There were false claims of an Ebola outbreak and a national emergency, but no emergency declaration was made., langchain_community.adapters.openai.convert_message_to_dict(message: BaseMessage) → dict [source] ¶. Convert a LangChain message to a dictionary. Parameters. message ( BaseMessage) - The LangChain message. Returns.


### Invoke with ToolCall

In [9]:
from langchain_core.messages import ToolCall

# A ToolCall is just a typeddict, so this is the same as saying tool.invoke({"args": {'query': 'weather in sf?'}, "type": "tool_call", "id": "foo", "name": "tavily"})
ans = tool.invoke(ToolCall(args={"query":"weather in sf?"},id="foo", name="tavily",type="tool_call"))
print(ans)

content='{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1722043273, \'localtime\': \'2024-07-26 18:21\'}, \'current\': {\'last_updated_epoch\': 1722042900, \'last_updated\': \'2024-07-26 18:15\', \'temp_c\': 15.2, \'temp_f\': 59.4, \'is_day\': 1, \'condition\': {\'text\': \'Sunny\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/113.png\', \'code\': 1000}, \'wind_mph\': 14.3, \'wind_kph\': 23.0, \'wind_degree\': 258, \'wind_dir\': \'WSW\', \'pressure_mb\': 1012.0, \'pressure_in\': 29.88, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 77, \'cloud\': 2, \'feelslike_c\': 14.5, \'feelslike_f\': 58.1, \'windchill_c\': 14.5, \'windchill_f\': 58.1, \'heatindex_c\': 15.2, \'heatindex_f\': 59.4, \'dewpoint_c\': 11.4, \'dewpoint_f\': 52.6, \'vis_km\': 10.0, \'vis_miles\': 6.0, \'uv\': 5.0, \'gust_mph\': 17.7, \'gust_kph\': 28.5

## Tool Return Objects

As part of our recent overhaul of tools, we are moving towards using a content/artifact paradigm for `ToolMessages`. The content will be a string and the artifact will contain the raw information of the tool call. 

In the case of the Tavily tool, our string is just a concatenation of the contents from each search result:

In [11]:
ans.content

'{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1722043273, \'localtime\': \'2024-07-26 18:21\'}, \'current\': {\'last_updated_epoch\': 1722042900, \'last_updated\': \'2024-07-26 18:15\', \'temp_c\': 15.2, \'temp_f\': 59.4, \'is_day\': 1, \'condition\': {\'text\': \'Sunny\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/113.png\', \'code\': 1000}, \'wind_mph\': 14.3, \'wind_kph\': 23.0, \'wind_degree\': 258, \'wind_dir\': \'WSW\', \'pressure_mb\': 1012.0, \'pressure_in\': 29.88, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 77, \'cloud\': 2, \'feelslike_c\': 14.5, \'feelslike_f\': 58.1, \'windchill_c\': 14.5, \'windchill_f\': 58.1, \'heatindex_c\': 15.2, \'heatindex_f\': 59.4, \'dewpoint_c\': 11.4, \'dewpoint_f\': 52.6, \'vis_km\': 10.0, \'vis_miles\': 6.0, \'uv\': 5.0, \'gust_mph\': 17.7, \'gust_kph\': 28.5}}, Log 

Now let's examine the artifact:

In [7]:
ans.artifact

{'query': 'weather in sf?',
 'follow_up_questions': None,
 'answer': 'The current weather in San Francisco is sunny with a temperature of 15.2°C (59.4°F). The wind speed is 23.0 kph coming from the WSW direction. The humidity is at 77%, and there is a visibility of 10.0 km (6.0 miles).',
 'images': ['https://www.sftourismtips.com/image-files/sf-weather-graphic-july.jpg',
  'https://www.weather2visit.com/images/charts/small/san-francisco-tmp-us.png',
  'http://www.cpc.ncep.noaa.gov/products/analysis_monitoring/regional_monitoring/clreat.gif',
  'https://cdn.hikb.at/charts/meteo-average-weather-weekly/sanfrancisco-meteo-average-weather-weekly.png?quality=5',
  'https://lh5.googleusercontent.com/T_I7_3pbFU3dQzN66OfxPFO7cMkgL-KDPergpTWqPAsnzuNvDOeKzX4gYNWxcnxnrZaC7UqulGi-e9w4Mbg8merKoSmcNoAtDB6zZdMJIS_boAdrzB__hBy_pYa03cyMEJdon1p1'],
 'results': [{'title': 'Weather in San Francisco',
   'url': 'https://www.weatherapi.com/',
   'content': "{'location': {'name': 'San Francisco', 'region': 'C

Inside our artifact we have access to the `results` key which provides us more specific information on each of the search results returned by our Tavily tool.

In addition we can see, the `answer`, `raw_content`, and `images` fields are populated because we set those parameters to true when we instantiated the tool. You can turn this off by setting the corresponding `include_answer`, `include_raw_content`, and `include_images` falgs to False. 